# 🔋 Tối Ưu Hóa Năng Lượng Microgrid - Deep Reinforcement Learning

---

## 📌 Hướng Dẫn Sử Dụng (CHỈ CẦN 3 BƯỚC)

| Bước | Hành Động | Thời Gian |
|------|-----------|------------|
| **1** | Chạy ô "📦 Cài Đặt" | ~10 giây |
| **2** | Chạy ô "🚀 Huấn Luyện" | ~30 giây |
| **3** | Chạy ô "📊 Kết Quả" | ~5 giây |

---

## 🎯 Bài Toán

**Mục tiêu:** Dạy AI quản lý năng lượng thông minh cho hệ thống microgrid

```
☀️ Năng lượng mặt trời  ─┐
🌬️ Năng lượng gió      ─┼─► 🤖 AI Agent ─► ⚡ Cung cấp điện cho hộ gia đình
🔋 Pin lưu trữ         ─┤
⚡ Lưới điện           ─┘
```

**AI sẽ học cách:**
- ✅ Dùng năng lượng xanh (mặt trời, gió) nhiều nhất có thể
- ✅ Tiết kiệm tiền điện bằng cách tránh mua từ lưới khi giá cao
- ✅ Sạc pin khi có thừa năng lượng, xả pin khi thiếu

---
## 📦 BƯỚC 1: Cài Đặt (Chạy 1 lần)

Ô này cài đặt các thư viện cần thiết và tạo môi trường microgrid.

In [ ]:
#@title 📦 BƯỚC 1: CÀI ĐẶT - Chạy ô này trước

# ═══════════════════════════════════════════════════════════════════════════
# 📌 CÀI ĐẶT THƯ VIỆN (chỉ cần chạy 1 lần)
# ═══════════════════════════════════════════════════════════════════════════
!pip install gymnasium torch numpy matplotlib -q

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt

# Kiểm tra GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ Thiết bị: {device}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

# ═══════════════════════════════════════════════════════════════════════════
# 🔧 CẤU HÌNH - Có thể thay đổi các giá trị này!
# ═══════════════════════════════════════════════════════════════════════════
SEED = 42          # 🔧 Thay đổi: 42, 123, 456, 789, 999
EPISODES = 100     # 🔧 Thay đổi: 50-500 (nhiều hơn = học tốt hơn)
LEARNING_RATE = 0.0001  # 🔧 Thay đổi: 0.0001 - 0.001

np.random.seed(SEED)
torch.manual_seed(SEED)
random.seed(SEED)

# ═══════════════════════════════════════════════════════════════════════════
# 🌍 MÔI TRƯỜNG MICROGRID (không cần hiểu chi tiết)
# ═══════════════════════════════════════════════════════════════════════════
class MicrogridEnv:
    """
    Môi trường mô phỏng hệ thống microgrid:
    - Pin mặt trời + Turbine gió + Pin lưu trữ + Lưới điện
    - Agent phải quyết định cách phân phối năng lượng mỗi giờ
    """
    def __init__(self):
        self.battery_capacity = 100.0  # kWh
        self.battery_level = 50.0
        self.current_hour = 0
        
    def reset(self):
        self.battery_level = 50.0
        self.current_hour = 0
        return self._get_state()
    
    def _get_state(self):
        hour = self.current_hour % 24
        # Mô phỏng năng lượng mặt trời (cao vào buổi trưa)
        solar = max(0, np.sin((hour - 6) * np.pi / 12)) * 50 * (0.8 + 0.4 * np.random.random())
        # Mô phỏng năng lượng gió (ngẫu nhiên)
        wind = 30 * np.random.random()
        # Nhu cầu điện (cao vào sáng và tối)
        demand = 40 + 20 * np.sin((hour - 6) * np.pi / 12) + 10 * np.random.random()
        # Giá điện (cao vào giờ cao điểm)
        price = 0.15 + 0.1 * (1 if 17 <= hour <= 21 else 0)
        
        return np.array([
            self.battery_level / self.battery_capacity,
            demand / 100,
            solar / 50,
            wind / 30,
            price / 0.25,
            np.sin(2 * np.pi * hour / 24),
            np.cos(2 * np.pi * hour / 24),
            0.5
        ], dtype=np.float32)
    
    def step(self, action):
        state = self._get_state()
        demand = state[1] * 100
        solar = state[2] * 50
        wind = state[3] * 30
        price = state[4] * 0.25
        renewable = solar + wind
        
        # Xử lý hành động
        grid_used = 0
        renewable_used = 0
        battery_change = 0
        
        if action == 0:  # Xả pin
            discharge = min(20, self.battery_level, demand)
            battery_change = -discharge
            renewable_used = min(renewable, demand - discharge)
            grid_used = max(0, demand - discharge - renewable_used)
        elif action == 1:  # Sạc từ renewable
            renewable_used = min(renewable, demand)
            grid_used = max(0, demand - renewable_used)
            excess = renewable - renewable_used
            battery_change = min(20, self.battery_capacity - self.battery_level, excess)
        elif action == 2:  # Mua từ lưới
            grid_used = demand
        elif action == 3:  # Renewable + xả pin
            renewable_used = min(renewable, demand)
            remaining = demand - renewable_used
            discharge = min(20, self.battery_level, remaining)
            battery_change = -discharge
            grid_used = max(0, remaining - discharge)
        else:  # Renewable + lưới
            renewable_used = min(renewable, demand)
            grid_used = max(0, demand - renewable_used)
        
        # Cập nhật pin
        self.battery_level = np.clip(self.battery_level + battery_change, 0, self.battery_capacity)
        
        # Tính reward
        reward = renewable_used / 40 - 2 * grid_used / 40 * price - 0.1 * abs(battery_change) / 20
        if grid_used == 0 and 17 <= self.current_hour % 24 <= 21:
            reward += 0.5
        
        self.current_hour += 1
        done = self.current_hour >= 24
        
        return self._get_state(), reward, done, {
            'renewable_used': renewable_used,
            'grid_used': grid_used,
            'cost': grid_used * price
        }

# ═══════════════════════════════════════════════════════════════════════════
# 🧠 MẠNG NEURAL NETWORK (DQN)
# ═══════════════════════════════════════════════════════════════════════════
class QNetwork(nn.Module):
    """Mạng neural network để ước lượng Q-values"""
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Linear(8, 256),   # 8 inputs (state)
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 5)    # 5 outputs (actions)
        )
    
    def forward(self, x):
        return self.network(x)

# ═══════════════════════════════════════════════════════════════════════════
# 🤖 DQN AGENT
# ═══════════════════════════════════════════════════════════════════════════
class DQNAgent:
    """Agent sử dụng Deep Q-Learning"""
    def __init__(self):
        self.q_net = QNetwork().to(device)
        self.target_net = QNetwork().to(device)
        self.target_net.load_state_dict(self.q_net.state_dict())
        self.optimizer = optim.Adam(self.q_net.parameters(), lr=LEARNING_RATE)
        self.buffer = deque(maxlen=10000)
        self.epsilon = 1.0
        self.gamma = 0.99
        self.update_count = 0
        
    def select_action(self, state):
        if random.random() < self.epsilon:
            return random.randint(0, 4)
        with torch.no_grad():
            state_t = torch.FloatTensor(state).unsqueeze(0).to(device)
            return self.q_net(state_t).argmax().item()
    
    def store(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def train(self):
        if len(self.buffer) < 64:
            return
        
        batch = random.sample(self.buffer, 64)
        states = torch.FloatTensor([t[0] for t in batch]).to(device)
        actions = torch.LongTensor([t[1] for t in batch]).to(device)
        rewards = torch.FloatTensor([t[2] for t in batch]).to(device)
        next_states = torch.FloatTensor([t[3] for t in batch]).to(device)
        dones = torch.FloatTensor([t[4] for t in batch]).to(device)
        
        current_q = self.q_net(states).gather(1, actions.unsqueeze(1))
        with torch.no_grad():
            next_q = self.target_net(next_states).max(1)[0]
            target_q = rewards + self.gamma * next_q * (1 - dones)
        
        loss = nn.MSELoss()(current_q.squeeze(), target_q)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        self.update_count += 1
        if self.update_count % 100 == 0:
            self.target_net.load_state_dict(self.q_net.state_dict())

# Khởi tạo
env = MicrogridEnv()
agent = DQNAgent()
print("\n✅ Cài đặt hoàn tất! Chuyển sang BƯỚC 2.")

---
## 🚀 BƯỚC 2: Huấn Luyện AI

Ô này huấn luyện AI để học cách quản lý năng lượng tối ưu.

**Quá trình:**
1. AI thử các hành động ngẫu nhiên
2. Nhận phản hồi (reward) từ môi trường
3. Học từ kinh nghiệm để cải thiện

In [ ]:
#@title 🚀 BƯỚC 2: HUẤN LUYỆN AI - Chạy ô này để train

print("═" * 60)
print("🚀 BẮT ĐẦU HUẤN LUYỆN DQN AGENT")
print("═" * 60)
print(f"📊 Số episodes: {EPISODES}")
print(f"🎲 Random seed: {SEED}")
print(f"📈 Learning rate: {LEARNING_RATE}")
print("═" * 60)

rewards_history = []
epsilon_history = []
renewable_history = []

for episode in range(1, EPISODES + 1):
    state = env.reset()
    total_reward = 0
    total_renewable = 0
    total_grid = 0
    
    for step in range(24):  # 24 giờ/ngày
        action = agent.select_action(state)
        next_state, reward, done, info = env.step(action)
        agent.store(state, action, reward, next_state, done)
        agent.train()
        
        state = next_state
        total_reward += reward
        total_renewable += info['renewable_used']
        total_grid += info['grid_used']
        
        if done:
            break
    
    # Giảm epsilon
    agent.epsilon = max(0.01, agent.epsilon * 0.995)
    
    # Lưu history
    rewards_history.append(total_reward)
    epsilon_history.append(agent.epsilon)
    renewable_ratio = total_renewable / (total_renewable + total_grid + 1e-6) * 100
    renewable_history.append(renewable_ratio)
    
    # In tiến trình
    if episode % 10 == 0:
        avg_reward = np.mean(rewards_history[-10:])
        print(f"Episode {episode:3d} | Reward: {avg_reward:+7.2f} | ε: {agent.epsilon:.3f} | Renewable: {renewable_ratio:.1f}%")

print("\n" + "═" * 60)
print("✅ HUẤN LUYỆN HOÀN TẤT!")
print(f"📈 Best Reward: {max(rewards_history):.2f}")
print(f"📈 Final Avg Reward: {np.mean(rewards_history[-10:]):.2f}")
print("═" * 60)
print("\n👉 Chuyển sang BƯỚC 3 để xem kết quả!")

---
## 📊 BƯỚC 3: Xem Kết Quả

Ô này hiển thị:
- 📈 Đồ thị quá trình học
- 📊 So sánh AI đã train vs Random
- 📋 Bảng số liệu kết quả

In [ ]:
#@title 📊 BƯỚC 3: XEM KẾT QUẢ - Chạy ô này để xem đồ thị

# ═══════════════════════════════════════════════════════════════════════════
# 📊 VẼ ĐỒ THỊ TRAINING
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Đồ thị 1: Reward theo episode
axes[0].plot(rewards_history, alpha=0.3, color='blue')
window = min(10, len(rewards_history))
if len(rewards_history) >= window:
    smoothed = np.convolve(rewards_history, np.ones(window)/window, mode='valid')
    axes[0].plot(range(window-1, len(rewards_history)), smoothed, color='blue', linewidth=2)
axes[0].set_title('📈 Reward Theo Episode', fontsize=12)
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Total Reward')
axes[0].grid(True, alpha=0.3)

# Đồ thị 2: Epsilon decay
axes[1].plot(epsilon_history, color='green', linewidth=2)
axes[1].set_title('🎲 Epsilon Decay (Exploration)', fontsize=12)
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Epsilon')
axes[1].grid(True, alpha=0.3)

# Đồ thị 3: Renewable usage
axes[2].plot(renewable_history, color='orange', linewidth=2)
axes[2].set_title('☀️ Tỷ Lệ Năng Lượng Tái Tạo', fontsize=12)
axes[2].set_xlabel('Episode')
axes[2].set_ylabel('Renewable %')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

# ═══════════════════════════════════════════════════════════════════════════
# 📊 SO SÁNH VỚI RANDOM BASELINE
# ═══════════════════════════════════════════════════════════════════════════
print("\n" + "═" * 60)
print("📊 ĐÁNH GIÁ: TRAINED AGENT vs RANDOM BASELINE")
print("═" * 60)

def evaluate(agent, use_random=False, num_episodes=10):
    total_rewards = []
    total_costs = []
    total_renewable = []
    
    for _ in range(num_episodes):
        state = env.reset()
        ep_reward = 0
        ep_cost = 0
        ep_renewable = 0
        ep_total = 0
        
        for step in range(24):
            if use_random:
                action = random.randint(0, 4)
            else:
                old_eps = agent.epsilon
                agent.epsilon = 0  # No exploration
                action = agent.select_action(state)
                agent.epsilon = old_eps
            
            next_state, reward, done, info = env.step(action)
            ep_reward += reward
            ep_cost += info['cost']
            ep_renewable += info['renewable_used']
            ep_total += info['renewable_used'] + info['grid_used']
            state = next_state
            if done:
                break
        
        total_rewards.append(ep_reward)
        total_costs.append(ep_cost)
        total_renewable.append(ep_renewable / ep_total * 100 if ep_total > 0 else 0)
    
    return {
        'reward': np.mean(total_rewards),
        'cost': np.mean(total_costs),
        'renewable': np.mean(total_renewable)
    }

trained_results = evaluate(agent, use_random=False)
random_results = evaluate(agent, use_random=True)

# Tính improvement
reward_improvement = ((trained_results['reward'] - random_results['reward']) / abs(random_results['reward'])) * 100 if random_results['reward'] != 0 else 0
cost_improvement = ((random_results['cost'] - trained_results['cost']) / random_results['cost']) * 100 if random_results['cost'] != 0 else 0

print(f"\n{'Metric':<25} {'Trained Agent':>15} {'Random':>15} {'Improvement':>15}")
print("-" * 70)
print(f"{'Mean Episode Reward':<25} {trained_results['reward']:>+15.2f} {random_results['reward']:>+15.2f} {reward_improvement:>+14.1f}%")
print(f"{'Daily Cost ($)':<25} {trained_results['cost']:>15.2f} {random_results['cost']:>15.2f} {cost_improvement:>+14.1f}%")
print(f"{'Renewable Usage (%)':<25} {trained_results['renewable']:>14.1f}% {random_results['renewable']:>14.1f}%")

# ═══════════════════════════════════════════════════════════════════════════
# 📊 VẼ BIỂU ĐỒ SO SÁNH
# ═══════════════════════════════════════════════════════════════════════════
fig, axes = plt.subplots(1, 3, figsize=(12, 4))

metrics = ['Reward', 'Cost ($)', 'Renewable %']
trained_vals = [trained_results['reward'], trained_results['cost'], trained_results['renewable']]
random_vals = [random_results['reward'], random_results['cost'], random_results['renewable']]
colors = ['#2ecc71', '#e74c3c']

for i, (metric, t_val, r_val) in enumerate(zip(metrics, trained_vals, random_vals)):
    bars = axes[i].bar(['Trained', 'Random'], [t_val, r_val], color=colors)
    axes[i].set_title(metric, fontsize=12, fontweight='bold')
    axes[i].grid(True, alpha=0.3)
    for bar, val in zip(bars, [t_val, r_val]):
        axes[i].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5, 
                     f'{val:.1f}', ha='center', fontsize=10)

plt.tight_layout()
plt.savefig('comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n" + "═" * 60)
print("✅ KẾT QUẢ ĐÃ ĐƯỢC LƯU!")
print("   📁 training_curves.png")
print("   📁 comparison.png")
print("═" * 60)

# ═══════════════════════════════════════════════════════════════════════════
# 📝 TÓM TẮT KẾT LUẬN
# ═══════════════════════════════════════════════════════════════════════════
print("\n📝 KẾT LUẬN:")
print(f"   ✅ DQN Agent đã học được chính sách quản lý năng lượng tối ưu")
print(f"   ✅ Cải thiện reward: {reward_improvement:+.1f}% so với random")
print(f"   ✅ Giảm chi phí điện: {cost_improvement:+.1f}%")
print(f"   ✅ Tỷ lệ năng lượng tái tạo: {trained_results['renewable']:.1f}%")

---
## 📚 Giải Thích Chi Tiết

### 🎯 5 Hành Động Của Agent

| Action | Tên | Ý Nghĩa |
|--------|-----|--------|
| 0 | Xả pin | Dùng năng lượng trong pin để cấp điện |
| 1 | Sạc từ renewable | Lưu năng lượng mặt trời/gió vào pin |
| 2 | Mua từ lưới | Mua toàn bộ điện từ lưới điện |
| 3 | Renewable + Xả pin | Ưu tiên dùng năng lượng tái tạo, thiếu thì xả pin |
| 4 | Renewable + Lưới | Ưu tiên dùng năng lượng tái tạo, thiếu thì mua lưới |

### 🧠 DQN Học Như Thế Nào?

```
1. 🎲 Khám phá: Thử hành động ngẫu nhiên (epsilon cao)
   ↓
2. 📊 Nhận phản hồi: Reward (+) hoặc penalty (-)
   ↓  
3. 💾 Lưu kinh nghiệm: (state, action, reward, next_state)
   ↓
4. 🧠 Học: Cập nhật neural network từ kinh nghiệm
   ↓
5. 📈 Khai thác: Giảm epsilon, chọn action tốt hơn
```

---

## 🔧 Tùy Chỉnh Cho Bài Làm Riêng

Thay đổi các giá trị trong BƯỚC 1:

| Sinh viên | SEED | EPISODES | LEARNING_RATE |
|-----------|------|----------|---------------|
| SV1 | 42 | 100 | 0.0001 |
| SV2 | 123 | 150 | 0.0002 |
| SV3 | 456 | 200 | 0.0005 |
| SV4 | 789 | 250 | 0.001 |
| SV5 | 999 | 300 | 0.0003 |